In [ ]:
# 在大多数情况下会失败或安装不兼容的版本，尤其是在 Windows 上使用 bitsandbytes 时。

# ❌ 为什么推荐用 pip 而不是 conda？
# transformers / accelerate / safetensors
# 在
# conda-forge
# 中有，但
# 版本通常滞后于 PyPI
# bitsandbytes
# Conda 官方频道不提供 Windows 版本
# ，
# conda install
# 会失败或回退到旧版/无 CUDA 支持版

# 💡 Hugging Face 官方明确推荐：在 Conda 环境中使用 pip 安装这些包（混合安装是安全的）。 

# ✅ 正确安装方式（推荐）
# 步骤 1：激活你的 Conda 环境
# bash


# 1
# conda activate your_env_name
# 步骤 2：用 pip 安装（关键！）
# bash


# 1
# 2
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# pip install transformers accelerate bitsandbytes safetensors
# ✅ 这样能确保： 

# bitsandbytes 安装的是 支持 CUDA 的最新版
# 所有包版本兼容（Hugging Face 测试基于 PyPI 版本）


In [ ]:
# conda config --add channels conda-forge
# conda config --set channel_priority strict
# onda install transformers accelerate bitsandbytes safetensors -y 

In [7]:
from huggingface_hub import login
login(new_session=False)

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# === 1. 配置 4-bit 量化（关键！）===
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


In [9]:
# === 2. 加载模型和 tokenizer ===
#model_id = "meta-llama/Llama-3.2-3B-Instruct"
model_id = "meta-llama/Llama-3.2-3B-Instruct-Quantized"  # ← HF 上的版本

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto",  # 自动使用 GPU
    trust_remote_code=False,
)


OSError: meta-llama/Llama-3.2-3B-Instruct-Quantized is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`

In [ ]:

# === 3. 构建对话 ===
messages = [
    {"role": "system", "content": "你是一个专业、友好的 AI 助手。"},
    {"role": "user", "content": "你好！请用中文写一首关于春天的诗。"}
]

# === 4. 生成回复 ===
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

outputs = model.generate(
    input_ids,
    max_new_tokens=256,      # 限制生成长度（防 OOM）
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)

# === 5. 打印结果 ===
response = outputs[0][input_ids.shape[-1]:]
print("\n🤖 回答：")
print(tokenizer.decode(response, skip_special_tokens=True))